In [1]:
# Dependencies
import os
import pandas as pd
from sqlalchemy import create_engine

,game_id,team,player_id,player_name
0,CHA202104080,0,kellb003,Brad Keller
1,CHA202104080,1,lynnl001,Lance Lynn
2,CHA202104110,0,minom001,Mike Minor
3,CHA202104110,1,ceasd001,Dylan Cease
4,CHA202104120,0,mcket001,Triston McKenzie
...,...,...,...,...
4853,PIT202110010,1,croww001,Wil Crowe
4854,PIT202110020,0,mahlt001,Tyler Mahle
4855,PIT202110020,1,kranm001,Max Kranick
4856,PIT202110030,0,sanmr001,Reiver Sanmartin
